In [4]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
import numpy as np
import time
import pandas as pd # Tablo çıktısı için
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


NUM_CLASSES = 4
BATCH_SIZE = 16
NUM_EPOCHS = 20     
LEARNING_RATE = 0.001
SEED = 42


DATA_DIR = r'C:\Users\ramis\Downloads\archive (1)\Multi-class Weather Dataset' 


torch.manual_seed(SEED)
if device.type == 'cuda':
    torch.cuda.manual_seed(SEED)

print(f"Kullanılan Cihaz: {device}")
print("Hücre 1 Tamamlandı.")

Kullanılan Cihaz: cpu
Hücre 1 Tamamlandı.


In [5]:

train_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Test/Val için Standart Dönüşüm
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(DATA_DIR, transform=None) 

toplam_veri = len(full_dataset)
train_size = int(0.6 * toplam_veri)
val_size = int(0.2 * toplam_veri)
test_size = toplam_veri - (train_size + val_size)

train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size], generator=torch.Generator().manual_seed(SEED)
)


train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = test_transform
test_dataset.dataset.transform = test_transform


dataloaders = {
    'train': DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0),
    'val': DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0),
    'test': DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
}

dataset_sizes = {'train': train_size, 'val': val_size, 'test': test_size}
class_names = full_dataset.classes

print(f"Veri Dağılımı -> Train: {train_size}, Val: {val_size}, Test: {test_size}")
print("Hücre 2 Tamamlandı.")

Veri Dağılımı -> Train: 675, Val: 225, Test: 225
Hücre 2 Tamamlandı.


In [6]:


class LeNet5_Improved(nn.Module):
    def __init__(self, num_classes=4):
        super(LeNet5_Improved, self).__init__()
        
        
        self.conv1 = nn.Conv2d(3, 6, 5) 
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        
        self.fc1 = nn.Linear(16 * 53 * 53, 120)
        self.dropout1 = nn.Dropout(0.5) 
        self.fc2 = nn.Linear(120, 84)
        self.dropout2 = nn.Dropout(0.5) 
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 53 * 53)
        
        x = F.relu(self.fc1(x))
        x = self.dropout1(x) 
        
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        
        x = self.fc3(x)
        return x


model_ft = LeNet5_Improved(num_classes=NUM_CLASSES).to(device)


optimizer_ft = optim.Adam(model_ft.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

print("Model: LeNet-5 (Improved) Hazır.")
print("Hücre 3 Tamamlandı.")

Model: LeNet-5 (Improved) Hazır.
Hücre 3 Tamamlandı.


In [7]:


def train_model(model, dataloaders, criterion, optimizer, num_epochs):
    best_model_wts = model.state_dict()
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        # Düzgün Epoch Sayacı (1'den başlatıldı)
        print(f'Epoch {epoch + 1}/{num_epochs}') 
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc.item()
                best_model_wts = model.state_dict()

    print(f'\nEn İyi Val Accuracy: {best_acc:.4f}')
    model.load_state_dict(best_model_wts)
    return model

print("Hücre 4 Tamamlandı.")

Hücre 4 Tamamlandı.


In [8]:
print("Eğitim Başlıyor...")
model_ft = train_model(model_ft, dataloaders, criterion, optimizer_ft, NUM_EPOCHS)
print("Eğitim Bitti.")

Eğitim Başlıyor...
Epoch 1/20
----------
train Loss: 1.1239 Acc: 0.4533
val Loss: 0.8746 Acc: 0.5822
Epoch 2/20
----------
train Loss: 0.8596 Acc: 0.5896
val Loss: 0.8198 Acc: 0.6489
Epoch 3/20
----------
train Loss: 0.7237 Acc: 0.6563
val Loss: 0.7004 Acc: 0.7022
Epoch 4/20
----------
train Loss: 0.7154 Acc: 0.6622
val Loss: 0.6356 Acc: 0.7556
Epoch 5/20
----------
train Loss: 0.6294 Acc: 0.7348
val Loss: 0.5386 Acc: 0.7778
Epoch 6/20
----------
train Loss: 0.6104 Acc: 0.7541
val Loss: 0.6343 Acc: 0.7778
Epoch 7/20
----------
train Loss: 0.5178 Acc: 0.7881
val Loss: 0.4914 Acc: 0.8089
Epoch 8/20
----------
train Loss: 0.4686 Acc: 0.8178
val Loss: 0.4758 Acc: 0.8222
Epoch 9/20
----------
train Loss: 0.3588 Acc: 0.8652
val Loss: 0.5335 Acc: 0.8178
Epoch 10/20
----------
train Loss: 0.4256 Acc: 0.8578
val Loss: 0.4187 Acc: 0.8400
Epoch 11/20
----------
train Loss: 0.3626 Acc: 0.8637
val Loss: 0.4637 Acc: 0.8489
Epoch 12/20
----------
train Loss: 0.2443 Acc: 0.9096
val Loss: 0.4212 Acc: 0

In [9]:


def evaluate_model_table(model, dataloader):
    model.eval()
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for inputs, labels in dataloader['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())
            
    
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    rec = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    
    results = pd.DataFrame({
        'Model': ['LeNet-5 (Improved)'],
        'Accuracy': [acc],
        'Precision': [prec],
        'Recall': [rec],
        'F1 Score': [f1]
    })
    
    return results, y_true, y_pred


print("\n--- TEST SONUÇLARI ---")
df_results, y_true, y_pred = evaluate_model_table(model_ft, dataloaders)


print(df_results.to_string(index=False, float_format="%.4f"))


print("\n--- Confusion Matrix (Doğru Sınıf vs Tahmin) ---")
print(confusion_matrix(y_true, y_pred))

print("\nHücre 6 Tamamlandı.")


--- TEST SONUÇLARI ---
             Model  Accuracy  Precision  Recall  F1 Score
LeNet-5 (Improved)    0.9244     0.9293  0.9244    0.9253

--- Confusion Matrix (Doğru Sınıf vs Tahmin) ---
[[47  1  0  7]
 [ 3 35  0  1]
 [ 1  0 79  2]
 [ 2  0  0 47]]

Hücre 6 Tamamlandı.
